In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import src.aformer_TF_gc_separated as aformer
#import src.aformer_TF as aformer
from src.layers.layers import *
import src.metrics as metrics
from src.optimizers import *
import src.schedulers as schedulers
import src.utils as utils

import training_utils_aformer_TF_genecentered_separated as training_utils


from scipy import stats

2022-11-17 20:24:00.502013: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-17 20:24:00.502052: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-4')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    #options.experimental_threading.max_intra_op_parallelism = 1
    mixed_precision.set_global_policy('mixed_bfloat16')
    tf.config.optimizer.set_jit(True)
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 12
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS

2022-11-17 20:24:02.546894: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-17 20:24:02.546946: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-17 20:24:02.546972: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


2022-11-17 20:24:02.824682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
2022-11-17 20:24:02.834389: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.113.28.194:8470}
2022-11-17 20:24:02.834443: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:60086}
2022-11-17 20:24:02.854478: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.113.28.194:8470}
2022-11-17 20:24:02.854534: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initial

INFO:tensorflow:Initializing the TPU system: node-4


INFO:tensorflow:Initializing the TPU system: node-4


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
with strategy.scope():

    train_steps=50#3200#5165 #320
    warmup_steps=10
    val_steps_h=10#3200#757 ### 5562
    val_steps_m=14
    num_epochs=5
    lr_base=0.001
    warmup_lr=1.0e-06

    data_it_tr_list = []
    data_it_val_list = []

    ### create dataset iterators
    heads_dict = {}
    data_dict_tr,data_dict_val,val_ho_it = training_utils.return_distributed_iterators("gs://picard-testing-176520/98k_genecentered_blacklist0.40_peaks_sep/preprocessed/",
                                                                                       "gs://picard-testing-176520/98k_genecentered_blacklist0.40_peaks_sep/val_holdout/preprocessed",
                                                                                       GLOBAL_BATCH_SIZE,
                                                                                       98304,
                                                                                       768,
                                                                                       196608,
                                                                                       128,
                                                                                       20,
                                                                                       4,
                                                                                       50,
                                                                                       strategy,
                                                                                       options)
    

In [4]:
with strategy.scope():
    inits = training_utils.get_initializers("/home/jupyter/dev/BE_CD69_paper_2022/enformer_fine_tuning/checkpoint/sonnet_weights")
    model = aformer.aformer(kernel_transformation = 'softmax_kernel_transformation',
                            dropout_rate = 0.25,
                            attention_dropout_rate = 0.15,
                            pointwise_dropout_rate=0.05,
                            input_length = 98304,
                            dim_reduce_length_seq = 768,
                            peaks_reduce_dim= 128,
                             num_heads = 4,
                             numerical_stabilizer = 0.001,
                             nb_random_features = 256,
                             hidden_size = 400,
                             shared_transformer_depth = 2,
                             pre_transf_channels = 400,
                             d_model = 400,
                             norm=True,
                             dim = 100, 
                             max_seq_length = 768,
                             use_rot_emb = True,
                             use_mask_pos = False, 
                             normalize = True,
                             seed = 3,
                             load_init=False,
                             inits=inits,
                             filter_list_seq=[192,224,256,288,320,384], 
                             freeze_conv_layers=False)
    

In [5]:
with strategy.scope():
    scheduler1= tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=2.0e-05,
        decay_steps=1000, alpha=1.0)
    scheduler1=WarmUp(initial_learning_rate=5.0e-05,
                                 warmup_steps=50,
                                 decay_schedule_fn=scheduler1)
    optimizer1 = tfa.optimizers.AdamW(learning_rate=scheduler1,
                                           weight_decay=5.0e-07)
    
    scheduler2= tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=2.0e-05,
    decay_steps=1000, alpha=1.0)
    scheduler2=WarmUp(initial_learning_rate=2.0e-05,
                                 warmup_steps=50,
                                 decay_schedule_fn=scheduler2)
    optimizer2 = tfa.optimizers.AdamW(learning_rate=scheduler2,
                                          weight_decay=5.0e-07)
    
    optimizers_in=optimizer1,optimizer2
    

In [6]:
with strategy.scope():
    metric_dict = {}
    train_step, val_step, val_ho_step,build_step, metric_dict = training_utils.return_train_val_functions(model,
                                                                                                          optimizers_in,
                                                                                                          strategy,
                                                                                                          metric_dict,
                                                                                                          150,
                                                                                                          150,
                                                                                                          150,
                                                                                                          GLOBAL_BATCH_SIZE,
                                                                                                          1.0,
                                                                                                          BATCH_SIZE_PER_REPLICA,
                                                                                                          loss_fn_main='poisson',
                                                                                                          use_peaks=True,
                                                                                                          use_atac=False,
                                                                                                          use_coef_var=False)


In [7]:
%matplotlib inline

In [ ]:
from scipy.stats.stats import pearsonr  
with strategy.scope():
    def sum_log(x):
        return np.log10(1.0 + np.nansum(x))
    
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(1, 15):
        start = time.time()
        if epoch_i == 1:
            build_step(data_dict_val)
            total_params = 0
            for k in model.trainable_variables:
                var = k.values[0]
                total_params += tf.size(var)
            print('total params: ' + str(total_params)) 
            
        train_step(data_dict_tr)

                   #data_dict_tr['mm'],
                   #data_dict_tr['rm'])
        print('hg_train_loss: ' + str(metric_dict['hg_tr'].result().numpy()))
        
        #print('hg_lr: ' + str(lr.numpy()))
        #print('hg_it: ' + str(it.numpy()))
        
        val_step(data_dict_val)
        val_ho_step(val_ho_it)

        val_losses.append(metric_dict['hg_val'].result().numpy())
        print('hg_val_loss: ' + str(metric_dict['hg_val'].result().numpy()))
        print('hg_val_pearson: ' + str(metric_dict['hg_corr_stats'].result()['pearsonR'].numpy()))
        print('hg_val_R2: ' + str(metric_dict['hg_corr_stats'].result()['R2'].numpy()))
        
        print('hg_val_pearson_ho: ' + str(metric_dict['hg_corr_stats_ho'].result()['pearsonR'].numpy()))
        print('hg_val_R2_ho: ' + str(metric_dict['hg_corr_stats_ho'].result()['R2'].numpy()))
        
        #print('hg_val_pearson_gene: ' + str(metric_dict['hg_corr_stats'].result()['feature_level_pearsonR'].numpy()))
        #print('hg_val_R2_gene: ' + str(metric_dict['hg_corr_stats'].result()['feature_level_R2'].numpy()))
        
        y_trues = metric_dict['hg_corr_stats'].result()['y_trues'].numpy()
        y_preds = metric_dict['hg_corr_stats'].result()['y_preds'].numpy()
        cell_types = metric_dict['hg_corr_stats'].result()['cell_types'].numpy()
        gene_map = metric_dict['hg_corr_stats'].result()['gene_map'].numpy()
        quit_bool=False
        if np.isnan(y_trues).any():
            quit_bool=True
            print('nan in trues')
        if np.isnan(y_preds).any():
            quit_bool=True
            print('nan in preds')
        if quit_bool:
            continue

        unique_preds = {}
        unique_trues = {}
        for k,x in enumerate(gene_map):
            unique_preds[(cell_types[k],x)] = y_preds[k]
            unique_trues[(cell_types[k],x)] = y_trues[k]
        
        unique_preds = dict(sorted(unique_preds.items()))
        unique_trues = dict(sorted(unique_trues.items()))
        
        
        print('overall gene pearsonsR:', pearsonr(y_trues,
                                                    y_preds)[0])
        
        print('overall gene spearmanR:', spearmanr(y_trues,
                                                    y_preds)[0])
        data = np.vstack([y_trues,y_preds])
        kernel = stats.gaussian_kde(data)(data)
        fig, ax = plt.subplots()
        sns.scatterplot(
            x=y_trues,
            y=y_preds,
            c=kernel,
            cmap="viridis",
            ax = ax)
        ax.set_xlim(0, max(y_trues))
        ax.set_ylim(0, max(y_trues))
        plt.show()
        
        
        ### now compute correlations across cell types
        across_cells_preds = {}
        across_cells_trues = {}
        
        for k,v in unique_preds.items():
            cell_t,gene_name = k
            if cell_t not in across_cells_preds.keys():
                across_cells_preds[cell_t] = []
                across_cells_trues[cell_t] = []
            else:
                across_cells_preds[cell_t].append(v)
                across_cells_trues[cell_t].append(unique_trues[k])
        cell_specific_corrs = []
        for k,v in across_cells_preds.items():
            trues = []
            preds = []
            for idx,x in enumerate(v):
                preds.append(x)
                trues.append(across_cells_trues[k][idx])
            try: 
                cell_specific_corrs.append(pearsonr(trues, 
                                                    preds)[0])
            except np.linalg.LinAlgError:
                continue
            except ValueError:
                continue
                
        sns.histplot(x=np.asarray(cell_specific_corrs), bins=50)
        plt.show()
        print('median_cell_crossgenes:', np.nanmedian(cell_specific_corrs))
                
            
        ### now compute correlations across genes
        across_genes_preds = {}
        across_genes_trues = {}
        
        for k,v in unique_preds.items():
            cell_t,gene_name = k
            if gene_name not in across_genes_preds.keys():
                across_genes_preds[gene_name] = []
                across_genes_trues[gene_name] = []
            else:
                across_genes_preds[gene_name].append(v)
                across_genes_trues[gene_name].append(unique_trues[k])
        genes_specific_corrs = []
        genes_specific_vars = []
        for k,v in across_genes_preds.items():
            trues = []
            preds = []
            for idx, x in enumerate(v):
                #if len(x) > 0:
                preds.append(x)
                trues.append(across_genes_trues[k][idx])
            try: 
                genes_specific_corrs.append(spearmanr(trues, 
                                                     preds)[0])
                genes_specific_vars.append(np.var(trues))
            except np.linalg.LinAlgError:
                continue
            except ValueError:
                continue
                
        sns.histplot(x=np.asarray(genes_specific_corrs), bins=50)
        plt.show()
        print('median_gene_crossdataset:', np.nanmedian(genes_specific_corrs))
            
        sns.scatterplot(
            x=genes_specific_vars,
            y=genes_specific_corrs)
        plt.show()

        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('duration(mins): ' + str(duration))
        print('patience counter at: ' + str(patience_counter))
        
        for key, item in metric_dict.items():
            item.reset_state()
        

total params: tf.Tensor(6190163, shape=(), dtype=int32)


In [ ]:
optimizer.lr(optimizer.iterations)

In [ ]:
strategy.reduce("SUM",
                optimizer.iterations, axis=None)